# Open inTRACKtive WITHIN a Jupyter notebook

### Dependencies

In [1]:
%env ANYWIDGET_HMR=1
import pandas as pd

from pathlib import Path
from IPython.display import clear_output
from intracktive.notebook_widget import Widget
from intracktive.convert import dataframe_to_browser, make_track_ids_consecutive

env: ANYWIDGET_HMR=1


INFO:watchfiles.main:1 change detected
INFO:watchfiles.main:1 change detected


### Load data

In [2]:
# df = pd.read_csv("https://public.czbiohub.org/royerlab/zoo/C_elegans/tracks.csv")
df = pd.read_csv(
    "https://public.czbiohub.org/royerlab/ultrack/inTRACKtive/2D_multicolor_tracks_with_hex.csv"
)
df = make_track_ids_consecutive(df)
df

,track_id,t,y,x,parent_track_id,CellColor
0,1,0,8.0,473.0,-1,8655811
1,1,1,9.0,470.0,-1,7347372
2,1,2,9.0,469.0,-1,7018912
3,1,3,9.0,469.0,-1,7542682
4,1,4,9.0,468.0,-1,7738780
...,...,...,...,...,...,...
96252,1390,297,1435.0,18.0,-1,3038725
96253,1390,298,1435.0,24.0,-1,939273
96254,1390,299,1432.0,15.0,-1,3092751
96255,1391,298,1432.0,1897.0,-1,65638


### Open inTRACKtive widget

In [3]:
zarr_data_url = dataframe_to_browser(df, Path(), flag_open_browser=False)
clear_output()

widget = Widget(dataset_url=zarr_data_url)
widget

INFO:root:Serving /var/folders/0x/59zp_92x3dsgz9ms3l24tbsm0000gp/T/tmphbmup5_l at http://127.0.0.1:8000
INFO:root:Server running...


Widget(dataset_url='http://127.0.0.1:8000/zarr_bundle.zarr/')

In [15]:
widget.get_selected_tracks

[5, 6, 7, 8, 9, 10, 11, 12]

In [71]:
# widget.select_tracks([152130])

widget.select_tracks([606, 995, 319, 466, 461])

In [72]:
def find_k_longest_distance_track(df, k):
    def calculate_start_end_distance(group):
        # Sort by time to get start and end points
        group = group.sort_values("t")
        start_point = group.iloc[0]
        end_point = group.iloc[-1]
        # Calculate Euclidean distance between start and end
        dx = end_point["x"] - start_point["x"]
        dy = end_point["y"] - start_point["y"]
        return (dx**2 + dy**2) ** 0.5

    # Calculate start-to-end distance for each track
    # Explicitly select the columns we need to avoid deprecation warning
    track_distances = df.groupby("track_id")[["x", "y", "t"]].apply(
        calculate_start_end_distance
    )
    return track_distances.nlargest(k).index.tolist()


def find_longest_track(df):
    track_durations = df.groupby("track_id")["t"].agg(lambda x: x.max() - x.min())
    return track_durations.idxmax()


def find_track_with_most_ancestors(parent_dict):
    def count_ancestors(track_id, visited=None):
        if visited is None:
            visited = set()

        # If we've seen this track before, we have a cycle
        if track_id in visited:
            return 0

        visited.add(track_id)

        # If track has no parent, return 0
        if track_id not in parent_dict:
            return 0

        # Count ancestors of parent + 1 for the parent itself
        return 1 + count_ancestors(parent_dict[track_id], visited)

    # Count ancestors for each track
    ancestor_counts = {
        track_id: count_ancestors(track_id) for track_id in parent_dict.keys()
    }

    # Find track with maximum ancestor count
    return max(ancestor_counts.items(), key=lambda x: x[1])[0]

In [73]:
filtered_df = df[df["parent_track_id"] != -1]
graph = dict(zip(filtered_df["track_id"], filtered_df["parent_track_id"]))

longest_track_id = find_longest_track(df)
print("longest", longest_track_id)

furthest_track_id = find_k_longest_distance_track(df, 5)
print("furthest", furthest_track_id)

most_ancestors_track_id = find_track_with_most_ancestors(graph)
print("ancestors", most_ancestors_track_id)  # 801

longest 129
furthest [606, 995, 319, 466, 461]
ancestors 809


In [29]:
df[df.track_id == 606]

,track_id,t,y,x,parent_track_id,CellColor,z
55916,606,56,1073.0,1202.0,604,10290051,0.0
55917,606,57,1073.0,1200.0,604,7734136,0.0
55918,606,58,1074.0,1200.0,604,8128378,0.0
55919,606,59,1060.0,1200.0,604,5637247,0.0
55920,606,60,1070.0,1195.0,604,6882140,0.0
...,...,...,...,...,...,...,...
56132,606,272,1425.0,1619.0,604,5570596,0.0
56133,606,273,1429.0,1617.0,604,5308466,0.0
56134,606,274,1431.0,1614.0,604,5308444,0.0
56135,606,275,1434.0,1609.0,604,4980754,0.0


In [34]:
print("\ngraph:")
for k, v in graph.items():
    if k < 20:
        print(k, v)


graph:
6 5
7 5
8 7
9 7
11 10
12 10
14 13
15 13


In [44]:
graph[809]

808

In [45]:
graph

{6: 5,
 7: 5,
 8: 7,
 9: 7,
 11: 10,
 12: 10,
 14: 13,
 15: 13,
 20: 19,
 21: 19,
 24: 23,
 25: 23,
 27: 26,
 28: 26,
 29: 28,
 30: 28,
 32: 31,
 33: 32,
 34: 32,
 35: 34,
 36: 34,
 37: 31,
 40: 39,
 41: 40,
 42: 41,
 43: 41,
 44: 40,
 45: 44,
 46: 44,
 47: 39,
 49: 48,
 50: 48,
 52: 51,
 53: 51,
 54: 53,
 55: 53,
 57: 56,
 58: 56,
 60: 59,
 61: 60,
 62: 60,
 63: 59,
 64: 63,
 65: 63,
 67: 66,
 68: 67,
 69: 67,
 70: 66,
 72: 71,
 73: 71,
 75: 74,
 76: 74,
 79: 78,
 80: 78,
 82: 81,
 83: 82,
 84: 82,
 85: 81,
 88: 87,
 89: 87,
 91: 90,
 92: 91,
 93: 91,
 94: 90,
 95: 94,
 96: 94,
 98: 97,
 99: 98,
 100: 98,
 101: 97,
 103: 102,
 104: 103,
 105: 103,
 106: 102,
 107: 106,
 108: 106,
 110: 109,
 111: 110,
 112: 111,
 113: 111,
 114: 110,
 115: 114,
 116: 114,
 117: 109,
 119: 118,
 120: 119,
 121: 120,
 122: 120,
 123: 119,
 124: 118,
 127: 126,
 128: 126,
 131: 130,
 132: 131,
 133: 131,
 134: 133,
 135: 133,
 136: 130,
 138: 137,
 139: 138,
 140: 139,
 141: 139,
 142: 138,
 143: 137,
 1